# Метод `__exit__`

В Python менеджеры контекста используют три аргумента в методе `__exit__`, которые называются `exc_type`, `exc_value` и `traceback`. Эти параметры передаются в `__exit__` метод, если в блоке `with` произошло исключение. Если исключения не возникло, все три аргумента будут None.

## Объяснение аргументов
* `exc_type`: Тип исключения, которое было поднято. Например, для ошибки деления на ноль это будет `ZeroDivisionError`.
* `exc_value`: Само исключение (экземпляр исключения), содержащее информацию о возникшей ошибке.
* `traceback`: Объект трассировки стека, показывающий точное место в коде, где возникло исключение. Этот объект можно использовать для подробного анализа ошибки, например, чтобы вывести трассировку.

Как `__exit__` обрабатывает исключения
* Если `__exit__` возвращает `True`, то исключение, возникшее в блоке `with`, будет подавлено и не выйдет за его пределы.
* Если `__exit__` возвращает `False` (или ничего не возвращает), исключение передаётся дальше, и блок `with` завершает работу с ошибкой.

## Пример использования exc_type, exc_value, traceback

Создадим менеджер контекста, который будет перехватывать все исключения, записывать их в лог и, в зависимости от типа исключения, решать, подавлять его или нет.

In [3]:
import traceback

class ExceptionLogger:
    def __init__(self, suppress_exceptions=()):
        self.suppress_exceptions = suppress_exceptions  # Список исключений для подавления

    def __enter__(self):
        # Возвращаем self, если он нужен внутри блока with
        return self

    def __exit__(self, exc_type, exc_value, exc_traceback):
        if exc_type is not None:
            # Печатаем тип исключения, его сообщение и трассировку
            print(f"Тип исключения: {exc_type.__name__}")
            print(f"Сообщение исключения: {exc_value}")
            print("Трассировка:")
            traceback.print_tb(exc_traceback)  # Вывод трассировки в консоль
            
            # Проверяем, нужно ли подавить это исключение
            if issubclass(exc_type, self.suppress_exceptions):
                print(f"Исключение '{exc_type.__name__}' подавлено.")
                return True  # Подавляем исключение
            else:
                print(f"Исключение '{exc_type.__name__}' не подавлено.")
                return False  # Исключение не подавляется и выбрасывается снова
        return False  # Возвращаем False, если исключения не было

In [4]:
with ExceptionLogger(suppress_exceptions=(ZeroDivisionError,)):
    print("Рассчитываем 1 / 0:")
    print(1 / 0)  # Подавленное исключение

with ExceptionLogger(suppress_exceptions=(ZeroDivisionError,)):
    print("\nРассчитываем 'abc' + 123:")
    print("abc" + 123)  # Исключение не будет подавлено

Рассчитываем 1 / 0:
Тип исключения: ZeroDivisionError
Сообщение исключения: division by zero
Трассировка:
Исключение 'ZeroDivisionError' подавлено.

Рассчитываем 'abc' + 123:
Тип исключения: TypeError
Сообщение исключения: can only concatenate str (not "int") to str
Трассировка:
Исключение 'TypeError' не подавлено.


  File "/var/folders/wm/khkwk2g178n7h8ngdvlc9htr0000gp/T/ipykernel_73725/258224984.py", line 3, in <module>
    print(1 / 0)  # Подавленное исключение
  File "/var/folders/wm/khkwk2g178n7h8ngdvlc9htr0000gp/T/ipykernel_73725/258224984.py", line 7, in <module>
    print("abc" + 123)  # Исключение не будет подавлено


TypeError: can only concatenate str (not "int") to str

Пояснение кода

1. Инициализация (`__init__`):
    * Принимает список исключений, которые должны быть подавлены, в параметре suppress_exceptions.
2.	Метод `__exit__`:
    * Если возникает исключение, его тип, сообщение и трассировка печатаются.
    * Проверяет, есть ли исключение в списке suppress_exceptions.
    * Если тип исключения входит в список подавляемых, возвращает True, и оно не выходит за пределы блока with.
    * Если тип исключения не входит в список, возвращает False, и исключение пробрасывается.